# Dataset Selection and Information

I chose to work with the **Intel Image Classification Dataset** available on Kaggle.

- **Dataset link**: [https://www.kaggle.com/datasets/puneet6060/intel-image-classification](https://www.kaggle.com/datasets/puneet6060/intel-image-classification)
- **Instances**: 25,000+ images
- **Classes**: 6 (Buildings, Forest, Glacier, Mountain, Sea, Street)
- **Image Size**: 150x150 pixels 
- **Target**: Multiclass classification  
  - `buildings`
  - `forest`
  - `glacier`
  - `mountain`
  - `sea`
  - `street`
